In [ ]:
#import single OTC company
stockfile=r'E:/daily/raw/20150525/3658.csv'
stockid='3658'
TimeId=20150525
if(os.path.exists(stockfile)):
    records=[]
    try:
        #read data from csv file
        with open(stockfile,'r') as dailyfile:
            reader =csv.reader(dailyfile)
            headline=reader.next()[0]
            #print headline.decode('big5')
            if headline=='***該股票該日無交易資訊***':
                print stockfile +"=***該股票該日無交易資訊***"
            reader.next()
            reader.next()
            
            for row in reader:
                record1={'id':'','broker':'','price':'','buy':'','sell':''}
                record2={'id':'','broker':'','price':'','buy':'','sell':''}
                #data=row.decode(code).split(',')
                record1['id']=row[0]
                record1['broker']=row[1][:4]
                record1['price']=float(row[2].replace(',',''))
                record1['buy']=int(row[3].replace(',',''))
                record1['sell']=int(row[4].replace(',',''))
                records.append(record1)
                #print record1               
                if len(row)>5 and len(row[6])>0:
                    record2['id']=row[6]
                    record2['broker']=row[7][:4]
                    record2['price']=float(row[8].replace(',',''))
                    record2['buy']=int(row[9].replace(',',''))
                    record2['sell']=int(row[10].replace(',',''))
                    records.append(record2)
                    #print record2
        
        #insert to DB
        cnx = mysql.connector.connect(user='threela', password='123456',
                                  host='10.120.31.4',
                                  database='threela')
        cursor = cnx.cursor()
        add_daily= ("INSERT INTO `threela`.`daily_tmp` "
                   "(StockId,BrokerId,TimeId,Buy,Sell,BuyPrice,SellPrice) "
                   "VALUES (%s,%s,%s,%s,%s,%s,%s)")
        data=[]
        for record in records:
           
            brokerid=record['broker']
            
            Buy=int(record['buy']/1000)
            Sell=int(record['sell']/1000)
            buyprice=0
            sellprice=0
            if record['buy'] >0:
                buyprice=record['price']
            if record['sell'] >0:
                sellprice=record['price']
            data_daily=(stockid,brokerid,TimeId,Buy,Sell,buyprice,sellprice)
            data.append(data_daily)
        
            cursor.executemany(add_daily, data)
            cnx.commit()
    except:
        print "Unexpected error:", stockfile,sys.exc_info()[0],sys.exc_info()[1]